In [1]:
import os

In [2]:
%pwd

'/Users/suyash/Desktop/Intent-classification-/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/suyash/Desktop/Intent-classification-'

In [31]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    train_path: Path
    tokenizer_name: Path
    


In [32]:
from src.ic.constants import *
from src.ic.utils.common import read_yaml,create_directories

In [33]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.paramss=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            train_path=config.train_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config



In [81]:
import os
from src.ic.logging import logging
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import load_from_disk

In [82]:
import pandas as pd

In [87]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        self.tokenizer=BertTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['intents'] )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['utterneces'])

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))


    

In [88]:
config=ConfigurationManager()
data_transformation_config=config.get_data_transformation_config()
data_transformation=DataTransformation(config=data_transformation_config)
data_transformation.convert()

Configuration Loaded: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/dataingestion', 'data_path': 'data/Bitext_Sample_Customer_Service_Training_Dataset/Training/Bitext_Sample_Customer_Service_Training_Dataset.csv', 'train_dir': 'artifacts/data_ingestion/train', 'test_dir': 'artifacts/data_ingestion/test'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'train_path': 'artifacts/data_ingestion/train.csv', 'tokenizer_name': 'DebertaTokenizerFast'}}
[2025-01-19 08:25:17,096: INFO: common: created directory at artifacts]
Data Transformation Config: {'root_dir': 'artifacts/data_transformation', 'train_path': 'artifacts/data_ingestion/train.csv', 'tokenizer_name': 'DebertaTokenizerFast'}
[2025-01-19 08:25:17,098: INFO: common: created directory at artifacts/data_transformation]


OSError: DebertaTokenizerFast is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`